## Setup


*   Perform imports and installs
*   Retrieve OpenAI API key
*   Define JSON format for output of translation text completion



In [ ]:
!pip install openai pyairtable

In [ ]:
import openai, json
from openai import OpenAI
from pydantic import BaseModel

from google.colab import userdata
openai.api_key = userdata.get("OPENAI_API_KEY")

class output(BaseModel):
  text: str

## Take user input


*   Text to translate and output language is hardcoded



In [ ]:
inputtext = """엄마 걱정
기형도
열무 삼십 단을 이고
시장에 간 우리 엄마
안 오시네, 해는 시든 지 오래
나는 찬밥처럼 방에 담겨
아무리 천천히 숙제를 해도
엄마 안 오시네, 배춧잎 같은 발소리 타박타박
안 들리네, 어둡고 무서워
금간 창 틈으로 고요히 빗소리
빈 방에 혼자 엎드려 훌쩍거리던
아주 먼 옛날
지금도 내 눈시울을 뜨겁게 하는
그 시절, 내 유년의 윗목"""

inputlanguage = "Korean"
outputlanguage = "English"

system_prompt = "You are a native speaker, who pays attention to translational nuance."

## Translation via text completion
* Perform text completion
* Print results

In [ ]:
client = OpenAI(api_key=openai.api_key)

def translatebasic(outputlang, inputtext, systemprompt):
    response = client.responses.create(
        model="gpt-4.1",
        input=f"Translate the following text to {outputlang}, keeping in mind the following instructions: {systemprompt}. Text: {inputtext}. ONLY RETURN THE TRANSLATION. ONLY THE TEXT-- do not thank me or ask questions- only text.",
    )
    final = response.output_text
    return(final)

translated_text = translatebasic(outputlanguage, inputtext, system_prompt)
print(translated_text)

In [ ]:
num_rounds = 3


def opposite_prompt(text: str, lang: str) -> str:
    """
    Return the semantic opposite of `text` (still in `lang`).
    """
    resp = client.responses.create(
        model="gpt-4o",
        input=[
            {"role": "system",
             "content": "Provide the semantic opposite of the user's text. "
                        "Return ONLY the opposite—no commentary."},
            {"role": "user", "content": text},
        ],
    )
    return resp.output_text.strip()

step_text = translatebasic(
    outputlanguage,
    inputtext,
    "You can speak every language as a native perfectly."
)
print("ROUND 0  (inputlanguage → outputlanguage)\n")
print(step_text, "\n")

for r in range(1, num_rounds + 1):
    back_to_input = translatebasic(
        inputlanguage,
        step_text,
        "Translate accurately, no extra words."
    )

    opposite_in_input = opposite_prompt(back_to_input, inputlanguage)
    print(f"ROUND {r}a  ➜  opposite in {inputlanguage}\n")
    print(opposite_in_input, "\n")

    step_text = translatebasic(
        outputlanguage,
        opposite_in_input,
        "Translate accurately, no extra words."
    )
    print(f"ROUND {r}b  ➜  back-translated to {outputlanguage}\n")
    print(step_text, "\n")